In [1]:
import pandas as pd
import geopandas as gpd
import io
import json
import os

os.chdir("/Users/canyonfoot/Documents/python_proj/nwf-process-geodata")
metadata_df = pd.read_csv("output_metadata.csv")

In [2]:
from src.download import DataDownloaderUploader
from src.process import Processor
from src.document import DataDocumenter
from src.export import s3Exporter

In [3]:
DataDownloaderUploader().download_and_upload_main()
Processor().process_raw_datasets()
DataDocumenter().document_processed_data()
s3Exporter().export_all("export_test")

29it [00:00, 36.14it/s]

Successfully uploaded data/raw/wildlife/mule_deer_migration_-_sublette/mule_deer.geojson
Successfully uploaded data/raw/wildlife/mule_deer_migration_-_platte_valley/mule_deer.geojson


33it [00:03,  5.19it/s]

Successfully uploaded data/raw/wildlife/mule_deer_migration_-_baggs/mule_deer.geojson


40it [00:09,  2.17it/s]

...

Processing data/raw/wildlife/sage_grouse/plss_sections_with_occupied_leks.zip


 98%|█████████▊| 62/63 [02:08<00:00,  1.23it/s]

Processing data/raw/wildlife/sage_grouse/winter_concentration_areas.zip


100%|██████████| 63/63 [02:08<00:00,  2.05s/it]


Downloaded data/processed/administrative/boundaries_and_management/wind_river_reservation.geojson to export_test/data/processed/administrative/boundaries_and_management/wind_river_reservation.geojson
Downloaded data/processed/crucial_critical/crucial_range/antelope.geojson to export_test/data/processed/crucial_critical/crucial_range/antelope.geojson
Downloaded data/processed/crucial_critical/crucial_range/bighorn_sheep.geojson to export_test/data/processed/crucial_critical/crucial_range/bighorn_sheep.geojson
Downloaded data/processed/crucial_critical/crucial_range/elk.geojson to export_test/data/processed/crucial_critical/crucial_range/elk.geojson
Downloaded data/processed/crucial_critical/crucial_range/moose.geojson to export_test/data/processed/crucial_critical/crucial_range/moose.geojson
Downloaded data/processed/crucial_critical/crucial_range/mule_deer.geojson to export_test/data/processed/crucial_critical/crucial_range/mule_deer.geojson
Downloaded data/processed/crucial_critical/c

In [7]:
s3Exporter().export_all("export_test")

Downloaded data/processed/crucial_critical/crucial_range/antelope.geojson to export_test/data/processed/crucial_critical/crucial_range/antelope.geojson
Downloaded data/processed/crucial_critical/crucial_range/bighorn_sheep.geojson to export_test/data/processed/crucial_critical/crucial_range/bighorn_sheep.geojson
Downloaded data/processed/crucial_critical/crucial_range/elk.geojson to export_test/data/processed/crucial_critical/crucial_range/elk.geojson
Downloaded data/processed/crucial_critical/crucial_range/moose.geojson to export_test/data/processed/crucial_critical/crucial_range/moose.geojson
Downloaded data/processed/crucial_critical/crucial_range/mule_deer.geojson to export_test/data/processed/crucial_critical/crucial_range/mule_deer.geojson
Downloaded data/processed/crucial_critical/crucial_range/rocky_mountain_goat.geojson to export_test/data/processed/crucial_critical/crucial_range/rocky_mountain_goat.geojson
Downloaded data/processed/crucial_critical/crucial_range/rocy_mountain

In [3]:
from fpdf import FPDF

# Create instance of FPDF class
pdf = FPDF()

# Add a page
pdf.add_page()

# Set font
pdf.set_font("Arial", size=12)

# Add a cell
pdf.cell(200, 10, txt="Welcome to PyFPDF!", ln=True, align='R')

# Save the PDF to a file
pdf.output("simple_demo.pdf")

''